## SVD with Python

---

### Basic SVD function

More Info: [How to Calculate the SVD from Scratch with Python](https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/)

In [1]:
#Import numpy and scipy
from numpy import array
from scipy.linalg import svd

In [2]:
# define a matrix
A = array([[1, 2], [3, 4], [5, 6]])
print(A)

# SVD
U, s, VT = svd(A)

[[1 2]
 [3 4]
 [5 6]]


In [3]:
print(U)
print(s)
print(VT)

[[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]
[9.52551809 0.51430058]
[[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]


### Use-case: Recommendation Engine

More Info: [Using Singular Value Decomposition to Build a Recommender System](https://machinelearningmastery.com/using-singular-value-decomposition-to-build-a-recommender-system/)

In [ ]:
import tarfile
import ast
import pandas as pd
import numpy as np
PATH = "deepyeti.ucsd.edu/jmcauley/datasets/librarything/lthing_data.tar.gz"

with tarfile.open(PATH) as tar:
    print("Files in tar archive:")
    tar.list()
    with tar.extractfile("lthing_data/reviews.json") as file:
        count = 0
        for line in file:
            count += 1
            if count > 3:
                break

In [5]:
# Collect records
reviews = []

with tarfile.open(PATH) as tar:
    with tar.extractfile("lthing_data/reviews.json") as file:
        for line in file:
            try:
                record = ast.literal_eval(line.decode("utf8"))
            except:
                print(line.decode("utf8"))
                raise
            if any(x not in record for x in ['user', 'work', 'stars']):
                continue
            reviews.append([record['user'], record['work'], record['stars']])
print(len(reviews), "records retrieved")

1387209 records retrieved


In [6]:
# Print a few sample of what we collected
reviews = pd.DataFrame(reviews, columns=["user", "work", "stars"])
reviews.head(5)

,user,work,stars
0,van_stef,3206242,5.0
1,dwatson2,12198649,5.0
2,amdrane2,12981302,4.0
3,Lila_Gustavus,5231009,3.0
4,skinglist,184318,2.0


In [7]:
# Look for the users who reviewed more than 50 books
usercount = reviews[["work","user"]].groupby("user").count()
usercount = usercount[usercount["work"] >= 50]

# Look for the books who reviewed by more than 50 users
workcount = reviews[["work","user"]].groupby("work").count()
workcount = workcount[workcount["user"] >= 50]

# Keep only the popular books and active users
reviews = reviews[reviews["user"].isin(usercount.index) & reviews["work"].isin(workcount.index)]
print("\nSubset of data:")
reviews.head(5)


Subset of data:


,user,work,stars
0,van_stef,3206242,5.0
6,justine,3067,4.5
18,stephmo,1594925,4.0
19,Eyejaybee,2849559,5.0
35,LisaMaria_C,452949,4.5


In [8]:
# Convert records into user-book review score matrix
reviewmatrix = reviews.pivot(index="user", columns="work", values="stars").fillna(0)
matrix = reviewmatrix.values

# Singular value decomposition
u, s, vh = np.linalg.svd(matrix, full_matrices=False)

# Find the highest similarity
def cosine_similarity(v,u):
    return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))

highest_similarity = -np.inf
highest_sim_col = -1
for col in range(1,vh.shape[1]):
    similarity = cosine_similarity(vh[:,0], vh[:,col])
    if similarity > highest_similarity:
        highest_similarity = similarity
        highest_sim_col = col

print("Column %d (book id %s) is most similar to column 0 (book id %s)" %
        (highest_sim_col, reviewmatrix.columns[col], reviewmatrix.columns[0])
)

Column 2092 (book id 9998) is most similar to column 0 (book id 10000)


In [9]:
# Inside Matrix 
reviewmatrix

work,10000,10001,1000167,10001797,10005525,10007394,10007399,10009,10012725,10012975,...,9978,9979582,9984,9986454,9989632,9989655,9989664,9993,9997232,9998
user,,,,,,,,,,,,,,,,,,,,,
,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Eva-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06nwingert,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1983mk,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1dragones,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zjakkelien,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zmagic69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zquilts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---